# Extraction des données ( via le package installé via "pip install pyspark")

# **Prérequis :
### Installer pip install pyspark via la gestion des dépendances de kaggle
### Attendre la fin du chargement du csv
### ???
### Profit !

### Objectif :
# Trier les données afin de récupérer celles qui nous interessent.

In [ ]:
import time
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when

In [ ]:
file_path_csv = "/kaggle/input/openfoodfacts/en.openfoodfacts.org.products.csv"
file_path_parquet = "/kaggle/working/en.openfoodfacts.org.products.parquet"

In [1]:
file_path_csv = "./data/en.openfoodfacts.org.products.csv"
file_path_parquet = "./data/en.openfoodfacts.org.products.parquet"

In [66]:
start_time = time.time()
print("Démarrage du script...")

# Initialiser une SparkSession avec des logs réduits
spark = SparkSession.builder \
    .appName("Exploration OpenFoodFacts") \
    .config("spark.sql.shuffle.partitions", "8") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")  # Réduction des logs

print("PySpark chargé")

try:
    # Charger le fichier CSV en tant que DataFrame Spark puis échantillonne 20%
    df_csv_before_sample = spark.read.csv(file_path_csv, header=True, inferSchema=True, sep="\t")
    print("Fichier CSV chargé.")
    df_csv = df_csv_before_sample.sample(withReplacement=False, fraction=0.2)  # Échantillonnage à 20%
    print("Echantillonage terminé")

    # Sauvegarder le DataFrame au format Parquet
    df_csv.write.parquet(file_path_parquet, mode="overwrite")
    print("Données sauvegardées au format Parquet.")

    # Charger le fichier Parquet pour une analyse future
    df_parquet = spark.read.parquet(file_path_parquet)
    print("Fichier Parquet chargé.")

    # Création de la table Hive
    print("Création et insertion dans la table Hive...")
    hive_table_start_time = time.time()

    # Écrire les données dans la table Hive
    df_csv.write.mode("overwrite").saveAsTable("hive_table")
    
except:
    print("ERRRRROOOOOOR")

finally:
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Temps d'exécution : {elapsed_time:.2f} secondes")


Démarrage du script...
PySpark chargé


Fichier CSV chargé.
Echantillonage terminé


Données sauvegardées au format Parquet.
Fichier Parquet chargé.
Création et insertion dans la table Hive...


Temps d'exécution : 277.31 secondes


In [67]:
# Mesure du temps pour compter les lignes du DataFrame CSV
csv_start_time = time.time()
csv_row_count = df_csv.count()  # Compter les lignes
csv_end_time = time.time()
csv_elapsed_time_ms = (csv_end_time - csv_start_time) * 1000
print(f"CSV (comptage des lignes): {csv_elapsed_time_ms:.3f} ms - Nombre de lignes: {csv_row_count}")

# Mesure du temps pour compter les lignes du DataFrame Parquet
parquet_start_time = time.time()
parquet_row_count = df_parquet.count()  # Compter les lignes
parquet_end_time = time.time()
parquet_elapsed_time_ms = (parquet_end_time - parquet_start_time) * 1000
print(f"Parquet (comptage des lignes): {parquet_elapsed_time_ms:.3f} ms - Nombre de lignes: {parquet_row_count}")

# Mesure du temps pour compter les lignes de la table Hive
hive_start_time = time.time()
df_hive = spark.sql("SELECT * FROM hive_table")  # Charger la table Hive
hive_row_count = df_hive.count()  # Compter les lignes
hive_end_time = time.time()
hive_elapsed_time_ms = (hive_end_time - hive_start_time) * 1000
print(f"Hive (comptage des lignes): {hive_elapsed_time_ms:.3f} ms - Nombre de lignes: {hive_row_count}")

# Comparaison des temps
print("\nComparaison des performances :")
print(f"CSV Execution Time: {csv_elapsed_time_ms:.3f} ms")
print(f"Parquet Execution Time: {parquet_elapsed_time_ms:.3f} ms")
print(f"Hive Execution Time: {hive_elapsed_time_ms:.3f} ms")

CSV (comptage des lignes): 20648.751 ms - Nombre de lignes: 702094
Parquet (comptage des lignes): 243.271 ms - Nombre de lignes: 702094
Hive (comptage des lignes): 390.368 ms - Nombre de lignes: 702094

Comparaison des performances :
CSV Execution Time: 20648.751 ms
Parquet Execution Time: 243.271 ms
Hive Execution Time: 390.368 ms


# Preliminary Analysis
### 1. Highlight the number of columns, rows, and the list of column names



In [69]:
# Mesurer le temps pour le DataFrame Parquet
parquet_start_time = time.time()
parquet_schema = df_parquet.schema  # Obtenir le schéma
parquet_columns = len(df_parquet.columns)  # Nombre de colonnes
parquet_rows = df_parquet.count()  # Nombre de lignes
parquet_elapsed_time_ms = (time.time() - parquet_start_time) * 1000

# Mesurer le temps pour le DataFrame CSV
csv_start_time = time.time()
csv_schema = df_csv.schema  # Obtenir le schéma
csv_columns = len(df_csv.columns)  # Nombre de colonnes
csv_rows = df_csv.count()  # Nombre de lignes
csv_elapsed_time_ms = (time.time() - csv_start_time) * 1000

# Afficher les résultats
print("\nRésultats pour le fichier Parquet:")

print(f"Nombre de colonnes (Parquet): {parquet_columns}")
print(f"Nombre de lignes (Parquet): {parquet_rows}")
print(f"Temps total d'exécution (Parquet): {parquet_elapsed_time_ms:.2f} ms")

print("\nRésultats pour le fichier CSV:")

print(f"Nombre de colonnes (CSV): {csv_columns}")
print(f"Nombre de lignes (CSV): {csv_rows}")
print(f"Temps total d'exécution (CSV): {csv_elapsed_time_ms:.2f} ms")

# Comparaison des temps
print("\nComparaison des performances:")
print(f"Temps d'exécution CSV: {csv_elapsed_time_ms:.2f} ms")
print(f"Temps d'exécution Parquet: {parquet_elapsed_time_ms:.2f} ms")
if csv_elapsed_time_ms < parquet_elapsed_time_ms:
    print("CSV est plus rapide.")
else:
    print("Parquet est plus rapide.")



Résultats pour le fichier Parquet:
Nombre de colonnes (Parquet): 206
Nombre de lignes (Parquet): 702094
Temps total d'exécution (Parquet): 239.64 ms

Résultats pour le fichier CSV:
Nombre de colonnes (CSV): 206
Nombre de lignes (CSV): 702094
Temps total d'exécution (CSV): 14932.49 ms

Comparaison des performances:
Temps d'exécution CSV: 14932.49 ms
Temps d'exécution Parquet: 239.64 ms
Parquet est plus rapide.


### .2 Handling Missing Values


In [56]:
from pyspark.sql.functions import col, count, when

start_time = time.time()

# Calculate missing data percentage for each column
total_rows = df_parquet.count() 
missing_data = (
    df_parquet.select([
        (count(when(col(c).isNull() | (col(c) == ""), c)) / total_rows).alias(c)
        for c in df_parquet.columns
    ])
)

# Transform columns into rows (melt operation)
missing_data_melted = missing_data.selectExpr(
    "stack({0}, {1}) as (Column, MissingPercentage)".format(
        len(df_parquet.columns),
        ", ".join([f"'{col}', `{col}`" for col in df_parquet.columns])
    )
).filter(col("MissingPercentage").isNotNull()).orderBy(col("MissingPercentage").desc())

# Identify columns with 100% missing data
columns_to_drop = (
    missing_data_melted.filter(col("MissingPercentage") == 1.0)
    .select("Column")
    .rdd.flatMap(lambda x: x)
    .collect()
)

# Drop columns with 100% missing values
df_cleaned = df_parquet.drop(*columns_to_drop)

# Display the top 10 columns with the highest missing percentages
print("Top 10 columns with the highest missing percentages:")
missing_data_melted.show(10, truncate=False)

# Print dropped columns
print(f"Columns dropped due to 100% missing values: {columns_to_drop}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Execution completed in {elapsed_time:.2f} seconds")


Top 10 columns with the highest missing percentages:


+-----------------------+------------------+
|Column                 |MissingPercentage |
+-----------------------+------------------+
|cities                 |1.0               |
|allergens_en           |1.0               |
|additives              |0.9999991461323352|
|nutrition-score-uk_100g|0.9999991461323352|
|elaidic-acid_100g      |0.9999980076421155|
|glycemic-index_100g    |0.9999977230195607|
|chlorophyl_100g        |0.9999974383970057|
|erucic-acid_100g       |0.9999968691518959|
|water-hardness_100g    |0.999996584529341 |
|caproic-acid_100g      |0.9999954460391214|
+-----------------------+------------------+
only showing top 10 rows

Columns dropped due to 100% missing values: ['cities', 'allergens_en']
Execution completed in 101.28 seconds


### 3. Handling Duplicates

In [58]:
from pyspark.sql.functions import col, count

start_time = time.time()

# Analyzing Duplicates in 'code', 'product_name', and 'brands'
duplicates = (
    df_parquet.groupBy("code", "product_name", "brands")
    .count()
    .filter(col("count") > 1)
)

# Affiche le nombre de doublons identifiés
print(f"There are {duplicates.count()} duplicate rows based on 'code', 'product_name', and 'brands'.")
duplicates.show(truncate=False)

# Remove duplicates where 'code', 'product_name', and 'brands' are the same
df_cleaned = df_parquet.dropDuplicates(["code", "product_name", "brands"])

print(f"Number of rows after removing duplicates: {df_cleaned.count()}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Execution completed in {elapsed_time:.2f} seconds")


There are 1968 duplicate rows based on 'code', 'product_name', and 'brands'.


+-----------------+------------------------------------------------------------+-----------------------------+-----+
|code             |product_name                                                |brands                       |count|
+-----------------+------------------------------------------------------------+-----------------------------+-----+
|3.245413820389E12|Sauce hollandaise                                           |Carrefour                    |2    |
|3.24541503968E12 |Ile de Beauté Rosé                                          |Réserve de Padulone,Carrefour|2    |
|3.245415141093E12|320G Beignet Chocolat X 4 Blis                              |Carrefour                    |2    |
|3.245677723693E12|Espresso Auchan Bio                                         |Auchan                       |2    |
|3.250390058991E12|Printiligne                                                 |Paturages                    |2    |
|3.250390109808E12|Chips nature                                 

Number of rows after removing duplicates: 3511425
Execution completed in 15.68 seconds


### 4. Handle outliers

In [60]:
from pyspark.sql.functions import regexp_extract

df_parquet = df_parquet.withColumn(
    "quantity_numeric",
    regexp_extract(col("quantity"), r"(\d+)", 1).cast("double")
)

numeric_columns = [
    field.name for field in df_parquet.schema.fields 
    if str(field.dataType) in ["IntegerType", "DoubleType", "FloatType"]
]
print(f"Numeric columns detected: {numeric_columns}")

if not numeric_columns:
    print("No numeric columns found. Please check your data.")
else:
    # Boucle sur les colonnes numériques pour détecter les outliers
    for column in numeric_columns:
        try:
            quantiles = df_parquet.approxQuantile(column, [0.25, 0.75], 0.1)
            if len(quantiles) < 2:
                print(f"Column '{column}' has insufficient data. Skipping...")
                continue
            
            q1, q3 = quantiles
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr

            print(f"Column: {column}")
            print(f"Q1: {q1}, Q3: {q3}, IQR: {iqr}")
            print(f"Lower Bound: {lower_bound}, Upper Bound: {upper_bound}")
            
            outliers = df_parquet.filter((col(column) < lower_bound) | (col(column) > upper_bound))
            print(f"Outliers detected in '{column}': {outliers.count()}")

        except Exception as e:
            print(f"Error processing column '{column}': {e}")


Numeric columns detected: []
No numeric columns found. Please check your data.


# Data cleaning


In [61]:
# display the schema of the cleaned DataFrame
df_parquet.describe()

In [ ]:
#select columns to keep
selected_column = [
    'code',
    'product_name',
    'brands',
    'categories',
    "main_category",
    'quantity',
    'packaging',
    'countries',
    'ingredients_text',
    'allergens',
    'serving_size',
    'energy-kcal_100g',
    'fat_100g',
    'saturated-fat_100g',
    "proteins_100g",
    'sugars_100g',
    'salt_100g',
    'nutriscore_score',
    'nutriscore_grade',
    "food_groups_en",
]

df_transformed = df_parquet.select(selected_column)
df_transformed.show(5, truncate=False)

In [ ]:
# convert the columns to the appropriate format
column_to_convert = ["quantity", "nutriscore_score", "energy-kcal_100g",
                     "fat_100g", "saturated-fat_100g", "proteins_100g", "sugars_100g", "salt_100g"]
# apply the conversion
for column in column_to_convert:
    df_transformed = df_transformed.withColumn(column, col(column).cast("double"))


In [ ]:
# display the schema of the transformed DataFrame
df_transformed.printSchema()

In [ ]:
# convert code in string
df_transformed = df_transformed.withColumn("code", col("code").cast("string"))

In [ ]:
# display the schema of the transformed DataFrame
df_transformed.printSchema()

# Transformation des données Transform :
Ajouter des colonnes calculées, par exemple : Indice de qualité nutritionnelle 
Calculer un score basé sur les nutriments (e.g., sodium, sugar, fiber). 
Extraire la catégorie principale d'un produit (e.g., "boissons", "snacks"). 
Regrouper les données par catégories (categories) pour analyser les tendances (e.g., moyenne des calories par catégorie).

--> Quel calcules effectuer ?  
--> Quel catégories créer ?


In [62]:
print("Transformation")

Transformation


# Analyse exploratoire :
Utiliser des fonctions de calcul sur fenêtre pour : 
Trouver les produits les plus caloriques par catégorie. 
Identifier les tendances de production par brands (marques). 
Générer des statistiques descriptives (e.g., médiane, moyenne des nutriments par catégorie

In [63]:
print("Exploration")

Exploration


# Sauvegarde des données Save :
Partitionner les données par catégories (categories) et années (year). 
Sauvegarder les résultats transformés en format Parquet avec compression Snappy. 
Sauvegarder les résultats transformés dans les bases de données: postgresql/sqlserver/mysql/Snowflake/BigQuery

In [64]:
print("Sauvegarde des données (load)")

Sauvegarde des données (load)




# Présentation des résultats :
Visualiser les résultats sous forme de graphiques ou tableaux 
(les étudiants peuvent utiliser un outil comme Jupyter Notebook en local ou Google Colab 

In [65]:
print("Présentation des données")

Présentation des données
